In [17]:
#%tensorflow_version 2.x
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import datasets, layers, models, callbacks
  
print(tf.version)

print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))


# Helper libraries
import numpy as np
import pandas as pd
#import matplotlib.pyplot as plt
import scipy.io as sio
from sklearn.model_selection import train_test_split
from sklearn.utils import compute_class_weight

from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from pathlib import Path
import os.path
#import datetime, os

from keras.applications import VGG16, VGG19, ResNet50, InceptionV3, InceptionResNetV2, Xception, MobileNet, DenseNet121, \
    NASNetMobile, EfficientNetB0, MobileNetV2, MobileNetV3Large
from tensorflow.python.keras.layers import Conv2D, Flatten, Dropout, Dense, MaxPooling2D


<module 'tensorflow._api.v2.version' from '/home/jarrad_foley135/miniconda3/envs/mlis_cw_car/lib/python3.9/site-packages/tensorflow/_api/v2/version/__init__.py'>
Num GPUs Available:  1


In [18]:
batch_size = 16
img_size = [240, 320]

df = pd.read_csv(r'machine-learning-in-science-ii-2023/training_norm.csv')
df['filename'] = df["image_id"].astype(str) + ".png"

df = df.sample(frac=1).reset_index(drop=True)
print(df)

x_train = df[0:int(len(df) * 0.6)]
x_validate = df[int(len(df) * 0.6):int(len(df) * 0.8)]
x_evaluate = df[int(len(df) * 0.8):]

# x_train = df[0:int(len(df) * 0.8)]
# x_validate = df[int(len(df) * 0.8):]


       image_id   angle  speed   filename
0         13428  0.7500    1.0  13428.png
1         12266  0.8750    1.0  12266.png
2         13479  0.3750    1.0  13479.png
3         11114  0.8125    1.0  11114.png
4          8010  0.5000    0.0   8010.png
...         ...     ...    ...        ...
14821       411  0.1250    0.0    411.png
14822     11552  0.6875    1.0  11552.png
14823     17464  0.5000    0.0  17464.png
14824      6216  0.6250    1.0   6216.png
14825     17558  0.5000    1.0  17558.png

[14826 rows x 4 columns]


In [19]:

shift = 0.1

training_datagen = ImageDataGenerator(
    rescale = 1./255,
    fill_mode='nearest',
    width_shift_range=shift,
    height_shift_range=shift
)

train_generator = training_datagen.flow_from_dataframe(
    dataframe=x_train,
    directory="machine-learning-in-science-ii-2023/training_data/training_data",
    x_col="filename",
    y_col=["angle","speed"],
    target_size=img_size,
    batch_size=batch_size,
    shuffle=False,
    class_mode='other')

#print(train_generator)

validation_datagen = ImageDataGenerator(rescale = 1./255)

val_generator = validation_datagen.flow_from_dataframe(
    dataframe=x_validate,
    directory="machine-learning-in-science-ii-2023/training_data/training_data",
    x_col="filename",
    y_col=["angle","speed"],
    target_size=img_size,
    shuffle=False,
    class_mode='other')

evaluate_datagen = ImageDataGenerator(rescale = 1./255)

eval_generator = evaluate_datagen.flow_from_dataframe(
    dataframe=x_evaluate,
    directory="machine-learning-in-science-ii-2023/training_data/training_data",
    x_col="filename",
    y_col=["angle","speed"],
    target_size=img_size,
    shuffle=False,
    class_mode='other')


Found 8895 validated image filenames.
Found 2965 validated image filenames.
Found 2966 validated image filenames.


In [20]:
shape = (*img_size, 3) # inherited image size with 3 color filters
input_shape = [240, 320, 3]
transfer = MobileNetV2(
    input_shape=input_shape, 
    include_top=False, 
    weights='imagenet',
    classifier_activation="relu"
)

for layer in transfer.layers[:-2]:
    layer.trainable = False

# CNN base
model = models.Sequential()

# model.add(layers.Conv2D(16, (3, 3), activation='relu', input_shape=shape))
# model.add(layers.BatchNormalization())
# model.add(layers.MaxPooling2D((2, 2)))

model.add(transfer)
model.build()
        
model.add(layers.Conv2D(128, (3, 3), strides= (2,2), activation='relu'))
model.add(layers.BatchNormalization())

# model.add(layers.Conv2D(64, (3, 3), activation='relu'))
# model.add(layers.BatchNormalization())
# model.add(layers.MaxPooling2D((2, 2), padding='same'))

model.add(layers.Flatten()),

model.add(layers.Dense(64, activation='relu'))
model.add(layers.BatchNormalization())
model.add(layers.Dropout(0.2))

# model.add(layers.Dense(16, activation='relu'))
# model.add(layers.BatchNormalization())
# model.add(layers.Dropout(0.2))

# Output layer
model.add(layers.Flatten())
model.add(layers.Dense(2,  activation='relu', kernel_initializer='normal'))

In [21]:
model.build()


In [22]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 mobilenetv2_1.00_224 (Funct  (None, 8, 10, 1280)      2257984   
 ional)                                                          
                                                                 
 conv2d_1 (Conv2D)           (None, 3, 4, 128)         1474688   
                                                                 
 batch_normalization_2 (Batc  (None, 3, 4, 128)        512       
 hNormalization)                                                 
                                                                 
 flatten_2 (Flatten)         (None, 1536)              0         
                                                                 
 dense_2 (Dense)             (None, 64)                98368     
                                                                 
 batch_normalization_3 (Batc  (None, 64)              

In [23]:
model.compile(
    optimizer=tf.optimizers.Adam(learning_rate=0.001),
    loss='mean_squared_error'
)

es = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',
    min_delta=0.001,
    patience=20,
    verbose=1,
    mode='min',
    restore_best_weights=True
)

history = model.fit(
    train_generator,
    batch_size=batch_size,
    validation_data=val_generator,
    callbacks=[es],
    epochs=100
)

Epoch 1/100
556/556 [==============================] - 106s 181ms/step - loss: 0.0926 - val_loss: 0.0269
Epoch 2/100
556/556 [==============================] - 99s 177ms/step - loss: 0.0318 - val_loss: 0.0203
Epoch 3/100
556/556 [==============================] - 100s 179ms/step - loss: 0.0223 - val_loss: 0.0129
Epoch 4/100
556/556 [==============================] - 101s 181ms/step - loss: 0.0198 - val_loss: 0.0150
Epoch 5/100
556/556 [==============================] - 101s 182ms/step - loss: 0.0187 - val_loss: 0.0110
Epoch 6/100
556/556 [==============================] - 107s 192ms/step - loss: 0.0166 - val_loss: 0.0111
Epoch 7/100
556/556 [==============================] - 106s 191ms/step - loss: 0.0168 - val_loss: 0.0122
Epoch 8/100
556/556 [==============================] - 104s 187ms/step - loss: 0.0159 - val_loss: 0.0130
Epoch 9/100
556/556 [==============================] - 114s 205ms/step - loss: 0.0148 - val_loss: 0.0301
Epoch 10/100
556/556 [==============================] - 

In [ ]:
filename = os.listdir("machine-learning-in-science-ii-2023/test_data/test_data") 

df = pd.DataFrame(filename)
df.columns = ["filename"]

df[['file', 'type']] = df.filename.str.split(".", expand = True)
df["file"] = df["file"].astype(str).astype(int)

df.sort_values(by=['file'], inplace=True)
df.reset_index(drop=True, inplace=True)
print(df)


test_datagen = ImageDataGenerator(rescale = 1./255)

test_images = test_datagen.flow_from_dataframe(
    dataframe=df,
    directory="machine-learning-in-science-ii-2023/test_data/test_data",
    x_col="filename",
    target_size=img_size,
    shuffle=False,
    class_mode=None)

print(test_images)

prediction = model.predict(test_images)
print(prediction.shape)
import math

a= prediction[:,0]
b= prediction[:,1]
b+=0.5
b= np.floor(b)

submissiondata={"angle":a , "speed":b} 
df = pd.DataFrame(submissiondata)
df.index += 1 
df.to_csv('Submission.csv')


      filename  file type
0        1.png     1  png
1        2.png     2  png
2        3.png     3  png
3        4.png     4  png
4        5.png     5  png
...        ...   ...  ...
1015  1016.png  1016  png
1016  1017.png  1017  png
1017  1018.png  1018  png
1018  1019.png  1019  png
1019  1020.png  1020  png

[1020 rows x 3 columns]
Found 1020 validated image filenames.
32/32 [==============================] - 5s 95ms/step
(1020, 2)


In [ ]:
from sklearn.metrics import classification_report

test_loss = model.evaluate(
    eval_generator,
    verbose=1
)

test_datagen = ImageDataGenerator(rescale = 1./255)

test_generator = test_datagen.flow_from_dataframe(
    dataframe=x_evaluate,
    directory="machine-learning-in-science-ii-2023/training_data/training_data",
    x_col="filename",
    target_size=img_size,
    shuffle=False,
    class_mode=None)


prediction = model.predict(test_generator, verbose =1)

a= prediction[:,0]
b= prediction[:,1]
b+=0.5
b= np.floor(b)

submissiondata={"angle":a , "speed":b} 
df = pd.DataFrame(submissiondata)
df.index += 1 
df.to_csv('evaluate.csv')



93/93 [==============================] - 7s 76ms/step - loss: 0.0096
Found 2966 validated image filenames.
93/93 [==============================] - 6s 62ms/step


In [ ]:
diff = abs(x_evaluate["angle"] - a)

results = pd.DataFrame({"Train":x_evaluate["angle"], "Predicted":a, "Difference":diff})
#results

#results.to_csv('evaluate.csv')

In [ ]:
results = pd.DataFrame({"Train":x_evaluate["speed"],"Predicted":b})
#results

#results.to_csv('evaluate.csv')

In [ ]:
model.save("model.hdf5")

In [ ]:
model.load_model("model.hdf5")
transfer.trainable = True

# for layer in transfer.layers[:2]:
#     layer.trainable = False
    
model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 mobilenetv2_1.00_224 (Funct  (None, 8, 10, 1280)      2257984   
 ional)                                                          
                                                                 
 conv2d (Conv2D)             (None, 3, 4, 128)         1474688   
                                                                 
 batch_normalization (BatchN  (None, 3, 4, 128)        512       
 ormalization)                                                   
                                                                 
 flatten (Flatten)           (None, 1536)              0         
                                                                 
 dense (Dense)               (None, 64)                98368     
                                                                 
 batch_normalization_1 (Batc  (None, 64)               2

In [ ]:

model.compile(
    optimizer=tf.optimizers.Adam(learning_rate=0.00001), # 1e-5 
    loss='mean_squared_error'
)

es = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',
    min_delta=0.0005,
    patience=30,
    verbose=1,
    mode='min',
    restore_best_weights=True
)

history = model.fit(
    train_generator,
    batch_size=batch_size,
    validation_data=val_generator,
    callbacks=[es],
    epochs=300
)


Epoch 1/300
556/556 [==============================] - 112s 167ms/step - loss: 0.0402 - val_loss: 0.0312
Epoch 2/300
556/556 [==============================] - 91s 163ms/step - loss: 0.0269 - val_loss: 0.0786
Epoch 3/300
556/556 [==============================] - 97s 174ms/step - loss: 0.0239 - val_loss: 0.0646
Epoch 4/300
556/556 [==============================] - 98s 176ms/step - loss: 0.0211 - val_loss: 0.0357
Epoch 5/300
556/556 [==============================] - 97s 175ms/step - loss: 0.0188 - val_loss: 0.0174
Epoch 6/300
556/556 [==============================] - 97s 175ms/step - loss: 0.0182 - val_loss: 0.0145
Epoch 7/300
556/556 [==============================] - 99s 178ms/step - loss: 0.0168 - val_loss: 0.0131
Epoch 8/300
556/556 [==============================] - 98s 176ms/step - loss: 0.0163 - val_loss: 0.0126
Epoch 9/300
556/556 [==============================] - 99s 178ms/step - loss: 0.0157 - val_loss: 0.0120
Epoch 10/300
556/556 [==============================] - 99s 178

In [ ]:
from sklearn.metrics import classification_report

test_loss = model.evaluate(
    eval_generator,
    verbose=1
)


93/93 [==============================] - 8s 82ms/step - loss: 0.0094


In [ ]:
model.save("model-fine-tuning.hdf5")